# Install Selenium
Selenium은 웹사이트 테스트를 위한 도구로 브라우저 동작을 자동화할 수 있습니다. 셀레니움을 이용하는 웹크롤링 방식은 바로 이점을 적극적으로 활용하는 것입니다. 프로그래밍으로 브라우저 동작을 제어해서 마치 사람이 이용하는 것 같이 웹페이지를 요청하고 응답을 받아올 수 있습니다.

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/c

# Import Library

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import time
import pandas as pd
import re
pd.set_option('display.max_colwidth', None)

# 전역변수
**sort:**
* 1 - 최신순
* 2 - 관련도순

**date_period:**
* all - 전체
* 1w - 최근 1주
* 1m - 최근 1달
* 1y - 최근 1년

**siteid:**
* www - 뉴스
* chosunbiz - 조선비즈
* 등등

**field:** paper - 지면뉴스만

**emd_word:** 포함할 단어  
**expt_word:** 제외할 단어

TITLE_OF_ARTICLE = [] : 기사 제목을 저장할 리스트  
DATE_OF_ARTICLE = [] : 기사 날짜를 저장할 리스트  
CONTENT_OF_ARTICLE = [] : 기사 내용을 저장할 리스트  
URL_OF_ARTICLE = [] : 기사의 URL을 저장할 리스트  


In [ ]:
TARGET_URL = 'https://www.chosun.com/search/query=%EC%9D%B8%EA%B6%8C&siteid=www&sort=1&date_period=1w&writer=&field=&emd_word=&expt_word=&opt_chk=false/'
# https://www.chosun.com/search/query=%EC%9D%B8%EA%B6%8C&siteid=www&sort=1&date_period=1w&writer=&field=&emd_word=&expt_word=&opt_chk=false/

TITLE_OF_ARTICLE = []
DATE_OF_ARTICLE = []
CONTENT_OF_ARTICLE = []
URL_OF_ARTICLE = []

**정적 웹 페이지(Static Web Page):**  
서버에 미리 저장된 파일이 그대로 전달되는 웹 페이지  
**동적 웹 페이지(Dynamic Web Page):**  
서버에 있는 데이터들을 스크립트에 의해 가공처리한 후 생성되어 전달되는 웹 페이지

조선일보의 경우 동적 웹 페이지 방식을 사용하고 있어서 bs4로 웹 페이지를 html을 받아올 경우 검색화면에서 보이는 타이틀이나 기사가 있는 링크를 가져오지 못하게 됩니다.  
그래서 셀레니움으로 더보기 버튼을 클릭하고 html을 bs4로 넘겨서 링크를 가져옵니다.  
셀레니움은 bs4보다 느리므로 셀레니움 + bs4를 사용하였습니다.

In [ ]:
def get_link_from_news_title(URL): 
    driver = webdriver.Chrome('chromedriver', options=options)
    driver.implicitly_wait(5) # implicity_wait은 뜻 그대로 브라우저에서 사용되는 엔진 자체에서 파싱되는 시간을 기다려 주는 메소드다.
    driver.get(URL)

    while True: # 반복문 break를 만나기 전까지 무한히 반복한다.
        try: # 더보기 버튼을 더 이상 누를수 없을 때 까지 반복한다.
            more_button = driver.find_element_by_xpath('//*[@id="load-more-stories"]') # 더보기 버튼 위치
            more_button.click() # 더보기 버튼 클릭
            time.sleep(1) # 페이지가 로딩되는데 시간이 필요하므로 1초의 텀을 준다.
        except: # 더보기 버튼을 더 이상 누를수 없을 때 실행된다.
            break # 반복문 종료

    soup = BeautifulSoup(driver.page_source, 'html.parser') # soup로 html 소스를 가져온다.
    
    for title in soup.select('div.story-card__headline-container.\|.box--margin-bottom-xs > h5'): # 기사들의 URL이 있는 링크에 접근해서 하나씩 title로 넘겨주는 반복문...h3이 나올수도 있음
        title_link = title.select('span')
        TITLE_OF_ARTICLE.append(title_link[0].get_text())
        #TITLE_OF_ARTICLE.append(title.span.text) 

        article_link = title.select('a') # URL이 위치하는 태그를 가리킨다.
        article_URL = 'https://www.chosun.com' + article_link[0]['href'] # URL 리스트에 저장한다.
        
        get_text(article_URL) # URL을 get_text 함수에 넘겨준다.

# get_text
- source_code_from_URL: 크룰링을 하기 위해 지정된 URL에 연결을 요청한다.
- soup: 데이터를 가져오기 위한 BeautifulSoup Library   
- title_of_article: 기사의 제목이 있는 태그를 가리킨다.
- date_of_article: 기사의 작성된 날짜가 있는 태그를 가리킨다.
- content_of_article: 기사의 내용이 있는 태그를 가리킨다.
- TITLE_OF_ARTICLE.append: 리스트에 기사의 제목를 저장한다.
- DATE_OF_ARTICLE.append: 리스트에 기사의 날짜를 저장한다.
- string_item: 태그를 제외한 string 형식으로 된 기사들을 가져온다.
- CONTENT_OF_ARTICLE.append: 기사의 내용을 리스트에 저장한다.

In [ ]:
def get_text(URL):
    driver = webdriver.Chrome('chromedriver', options=options)
    driver.implicitly_wait(5)
    driver.get(URL)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    date_of_article = soup.select('div.article-dateline.\|.flex.flex--justify-space-between.flex--align-items-center.box--border.box--border-grey-40.box--border-horizontal.box--border-horizontal-bottom.box--pad-bottom-sm > span')
    DATE_OF_ARTICLE.append(date_of_article[0].string)

    content_of_article = soup.select('section.article-body')
    string_item = str(content_of_article[0].find_all(text=True))
    CONTENT_OF_ARTICLE.append(string_item) 

In [ ]:
get_link_from_news_title(TARGET_URL)

In [ ]:
df = pd.DataFrame(list(zip(TITLE_OF_ARTICLE, CONTENT_OF_ARTICLE, DATE_OF_ARTICLE)), columns =['Title', 'Content', 'Date']) 

In [ ]:
df.head()

,Title,Content,Date
0,찬성 2895 반대 0표...홍콩 장악 끝낸 중국의 전인대 투표,"['중국이 홍콩의 행정장관과 입법회(의회) 의원을 친중(親中) 인사로 뽑도록 바꾼 홍콩 선거제도 개편안을 11일 폐막한 전국인민대표대회(전인대·국회 격)에서 확정했다. 중국에 충성하는 ‘애국자’만 홍콩의 선출직 공무원이 될 수 있다는 핵심 원칙이 담겼다. 이에 따라 홍콩에 대한 중국의 지배력이 극대화될 전망이다.', '중국 전인대는 이날 찬성 2895명, 반대 0명, 기권 1명으로 홍콩 행정장관과 입법회 의원 선거제도 개편안을 통과시켰다. 중국 관영 신화통신에 따르면 개편안에는 홍콩 선거위원회(행정장관 선거인단) 위원 수를 현재 1200명에서 1500명으로 확대하는 내용이 담겼다. 행정장관 입후보 자격을 강화하는 동시에 내년 캐리 람 현 행정장관의 후임 선출을 앞두고 선거인단에 친중 인사들을 추가로 확보하려는 조치로 해석된다.', '홍콩 정부가 후보자격심사위원회를 만들어 홍콩 행정장관 후보와 입법회 의원 후보의 자격을 심사하는 내용도 신설됐다. 위원회는 후보가 홍콩기본법, 홍콩 국가보안법, 전인대 요구 사항 등에 적합한지 심사해 출마 여부를 결정한다. 이럴 경우, 중국 정부가 원하지 않는 인물이 행정장관이나 입법회 선거에 출마하는 게 사실상 불가능해질 전망이다.', '시진핑 중국 국가주석(가운데)이 11일 베이징 인민대회당에서 열린 전국인민대표대회 전체회의에서 홍콩 선거제 개편안이 통과되지 박수를 치고 있다./AFP 연합뉴스', '이번 전인대는 또 홍콩 입법회 의원 수를 70명에서 90명으로 늘리기로 했다. 현재는 입법회 의원 중 절반인 35명은 홍콩 시민이 직접 선거로 뽑고 나머지는 직능 단체 간접 선거로 선출한다. 하지만 앞으로는 친중 성향 인사들이 다수를 차지하고 있는 선거위원회도 입법회 의원 일부를 선출하도록 했다. 구체적 비율은 공개되지 않았지만 직접 선거로 뽑는 의원 비율은 줄어들 것으로 보인다. 앞으로는 반중 성향인 홍콩 야권이 입법회 선거에서 대승을 거두더라도 홍콩 의회를 좌지우지할 수 없다는 의미다.', '전인대는 이날 통과된 개편 방안에 대해 “국가와 홍콩을 사랑하는 사람이 홍콩을 다스리는 원칙을 확고히 하고 국가 주권과 안전, 발전 이익을 지키는 데 유리하다”고 했다. 시진핑 주석을 비롯해 3000명 가까운 전인대 대표들은 이날 전인대 폐막식 겸 전체회의에서 예산안 등 10개 안건을 처리했는데 홍콩 선거제 개편안이 통과됐을 때 박수가 가장 오랫동안 이어졌다고 한다. 전인대에서는 안건 대부분이 압도적 찬성으로 통과되지만 지난해 홍콩보안법 투표 때는 기권 6표, 반대 1표가 있었다. 반면 이번 선거제 개편안은 반대 없이 기권 1표만 나왔다.', '홍콩 야권은 이번 선거제 개편에 대해 “범민주파에 대한 정치적 사망 선고” “일국양제의 끝”이라고 반발했다. 홍콩 사우스차이나모닝포스트는 “후보자격심사위원회가 입법회 의원 등 선출직 공무원 후보를 심사하게 되면서 당국이 충분히 애국적이라고 생각하지 않는 야당 후보는 사실상 출마가 금지될 것”이라고 했다. 반면 전인대 홍콩 대표인 훙웨이민(洪爲民) 홍콩인터넷협회 명예회장은 본지 통화에서 “홍콩 정부를 비판하지 말라는 뜻이 아니다”라며 “애국적인 야당도 충분히 가능하고 이미 일부 그런 움직임도 나타나고 있다”고 말했다.', '홍콩 언론들은 이번 선거제 개편에 대해 1997년 홍콩 반환 이후 가장 큰 폭의 정치적 개편(shake up)이라고 평가했다. 중국은 2019년 홍콩에서 6개월간 대규모 반중 시위가 이어지자 지난해 홍콩 내 반중 세력을 감시·처벌하는 내용의 홍콩보안법을 제정했다. 특히, 시진핑 중국 국가주석이 ‘애국자치항(愛國者治港·애국자가 홍콩을 다스린다)’ 원칙을 강조하면서 홍콩 정치권에 대한 개편 작업이 더욱 탄력을 받았다.', '중국이 홍콩 선거제 개편을 결정하면서 미국과의 갈등 관계가 더욱 악화될 것이란 전망이 나온다. 지난해 중국이 홍콩보안법을 제정하자 도널드 트럼프 당시 미 대통령은 무역 등에서 홍콩에 대한 특별 대우를 철회하고 중국과 홍콩 주요 인사들을 제재 명단에 올렸다. 지난 1월 취임한 조 바이든 행정부는 외교 정책 측면에서 트럼프 행정부보다 민주주의와 인권을 더 강조하고 있다. 미국은 홍콩 선거제 개편 결정 전날인 10일 미사일 구축함인 존핀함을 대만해협에 보내고 11일 소셜미디어를 통해 사진을 공개했다. 이에 대해 중국군 동부전구 장춘후이(張春暉) 대변인은 “미 군함의 행위는 잘못된 신호를 보내 의도적으로 지역 정세를 해치고 대만해협의 평화와 안정을 훼손한다”며 비판 성명을 냈다.', '한편 리커창 총리는 전인대 폐막 후 열린 내외신 기자회견에서 “중국이 올해 성장 목표를 작년 대비 6% 이상으로 잡았지만 해외 기관들은 8%까지 보고 있다”는 질문에 대해 “6% 이상이라는 경제성장 목표는 낮은 게 아니다”라며 “안정적으로 가야 유리하게 갈 수 있다”고 했다. 리 총리는 미·중 관계에 대해 “양국 사이에는 공통 이익이 있고 이 공통점에 더 많은 힘을 쏟아야 한다”며 “공통 이익을 확대하고 중·미 관계가 어려움을 넘어 전체적으로 안정적인 방향으로 가도록 노력해야 한다”고 했다.']",입력 2021.03.11 21:49 | 수정 2021.03.11 21:49
1,형제복지원 수사 검사 “원장 무죄는 대법원 집단무결주의 때문”,"['대법원 2부(주심 안철상 대법관)이 특수감금 등 혐의로 기소된 형제복지원장 고 박모씨에 대한 비상상고를 기각한 11일 오전 서울 서초구 대법원 법정 앞에서 형제복지원 피해자 및 가족들이 오열하고 있다. /뉴시스', '부산 형제복지원 원장에 대한 무죄 판결을 파기해달라는 검찰 비상상고를 대법원이 11일 받아들이지 않은 것과 관련, 이 사건을 처음 폭로했던 수사 검사가 “대법원의 집단 무결주의 때문”이라며 강하게 비판했다.', '김용원 변호사는 이날 페이스북에 ‘형제복지원 사건 비상상고심에 대한 수사검사의 소회’라는 글을 통해 대법원 결정을 비판했다. 그는 1987년 부산지검 울산지청 평검사 시절 이 문제를 처음으로 제기한 수사검사로, 1970~80년대 부랑자들을 상대로 끔찍한 인권유린 행위를 한 형제복지원 원장 고(故) 박인근씨를 특수감금 등 혐의로 구속기소했었다. 그는 “당시 전두환 정권의 법률적 들러리였던 대법관들이 무죄를 선고하였던 것인데, 이번에도 대법원은 그 무죄 판결을 유지했다”며 “나는 그것이 대법관들의 집단 무결주의 때문이라고 판단한다”고 했다. 대법원 스스로 대법원 판결의 오류를 인정하려들지 않았기 때문이라는 뜻이다. 김 변호사는 “대법관들은 군사정권 시절 이래 지금까지 숱한 과오를 저질러 왔지만 한번도 자신들의 과오를 시인하거나 반성한 적이 없다”고 지적했다.', '그러면서 “잘못된 대법원 판결이 있었다고 해도 그것은 수사기관의 잘못된 수사 때문이라고 언제나 책임을 전가했는데, 형제복지원 사건은 그런 방식이 불가능한 사건”이라며 “그래서 사건을 얼버무려 특수감금 부분의 무죄를 유지한 것”이라고 지적했다. 김 변호사는 “이 나라는 검찰개혁보다는 사법부 개혁이 훨씬 절실하다는 나의 오래된 소신을 다시 한 번 씁쓸하게 확인한다”고 했다.', '이날 대법원 2부(주심 안철상 대법관)는 형제복지원 원장의 특수감금 혐의를 무죄로 판단한 1989년 대법원 무죄 판결을 다시 판단해 달라는 검찰의 비상상고를 기각했다. 비상상고 요건에 맞지 않다는 것이다. 비상상고는 사실관계를 제외하고, 법원이 적용한 법령이 잘못된 경우 인정되는데 당시 대법원이 형법 20조(정당 행위)를 적용한 것에 문제가 없다는 것이다. 앞서 검찰은 비상상고를 신청하며 ‘내무부 훈령(부랑자들에 대한 관리 지침)이 위법했다’고 했지만 이 법령은 형법 20조 정당행위로 판단하기 위한 자료였을 뿐이라는 것이다. 재판부는 “(대법원 비상상고가 아니라) 정부의 적절한 조치를 통해 피해자들의 아픔이 치유되어 사회통합이 실현되기를 기대한다”고 했다. 작년 국회에서 통과된 과거사정리 기본법을 통해 사건을 재조사하고, 이를 통해 피해자와 유가족들의 명예회복이 이뤄지길 바란다는 것이다.', '부랑인을 선도한다는 명분으로 1975년 설립된 형제복지원은 약 12년간 일종

In [ ]:
df.to_csv("df.csv",index=True, encoding='utf-8-sig')